In [32]:
import json
from tqdm import tqdm
import os
import re

import json
import dask.bag as db
from dask import dataframe as dd

In [14]:
# lets know where we are
cwd = os.getcwd()
print(cwd)

/Users/bajayi-obe/Documents/personal_git/cvd19-truth-finder/notebooks


# Retrieiver

In [15]:
RAW_DOC_PATH_BIORXIV_MEDRXIV = "../cvd19-documents/biorxiv_medrxiv/biorxiv_medrxiv"
RAW_DOC_PATH_COMM_USE_SUBSET = "../cvd19-documents/comm_use_subset/comm_use_subset"
RAW_DOC_PATH_CUSTOM_LICENSE = "../cvd19-documents/custom_license/custom_license"
RAW_DOC_PATH_NONCOMM_USE_SUBSET = "../cvd19-documents/noncomm_use_subset/noncomm_use_subset"

PROCESSED_DOC_PATH = "../data/covid19/pre-database-documents/"

# Create Pre-Processed Files from papers

In [16]:
def iter_files(path):
    """Walk through all files located under a root path."""
    if os.path.isfile(path):
        yield path
    elif os.path.isdir(path):
        for dirpath, _, filenames in os.walk(path):
            for f in filenames:
                yield os.path.join(dirpath, f)
    else:
        raise RuntimeError('Path %s is invalid' % path)
        
def collect_text(text):
    """Combing list of text elements into one."""
    return " ".join(regex_filter(seg['text']) for seg in text)

def regex_filter(text):
    """Removing numbers and puncutation."""
    text = re.sub(r" \d+", "",text)
    return re.sub(r"[^A-Za-z0-9 -]+", "",text)

def clean_json_read(json_file_name):
    """Convert multi-line json files into one large file of single line jsons."""
    
    # reading in document
    document = open(json_file_name).read()

    # removing new lines and whitespace
    document = document.replace("\n", "")
    
    document_obj = json.loads(document)

    return document_obj

### Collecting jsons into single files for easier processing

In [17]:
# get all json papers in directory
BIORXIV_MEDRXIV_JSON_PATH_LIST = list(iter_files(RAW_DOC_PATH_BIORXIV_MEDRXIV))
COMM_USE_SUBSET_JSON_PATH_LIST = list(iter_files(RAW_DOC_PATH_COMM_USE_SUBSET))
CUSTOM_LICENSE_JSON_PATH_LIST = list(iter_files(RAW_DOC_PATH_CUSTOM_LICENSE))
NONCOMM_USE_SUBSET_JSON_PATH_LIST = list(iter_files(RAW_DOC_PATH_NONCOMM_USE_SUBSET))

JSON_DICT = {"biorxiv_medrxiv": BIORXIV_MEDRXIV_JSON_PATH_LIST,
             "comm_use_subset": COMM_USE_SUBSET_JSON_PATH_LIST,
             "comm_license": CUSTOM_LICENSE_JSON_PATH_LIST,
             "noncomm_use_subset": NONCOMM_USE_SUBSET_JSON_PATH_LIST}

In [18]:
# collecting
def combine_all_jsons(category):
    json_file_name = JSON_DICT[category]
    with open(f"../data/covid19/pre-database-documents/{category}.json", "w") as f:
        for json_path in tqdm(json_file_name):    
            f.write(json.dumps(clean_json_read(json_path)) + "\n")

In [20]:
combine_all_jsons("biorxiv_medrxiv")

combine_all_jsons("comm_use_subset")

combine_all_jsons("comm_license")

combine_all_jsons("noncomm_use_subset")

100%|██████████| 2350/2350 [00:06<00:00, 339.23it/s]


## Collecting all json text's

In [21]:
d_json_files = db.read_text("../data/covid19/pre-database-documents/*.json").map(json.loads)

In [22]:
def collect_text(record):
    
    return {"paper_id": record["paper_id"],
            "title": record["metadata"]["title"],
            "abstract": [abst["text"] + abst["section"]
                         for abst in record["abstract"]],
            "body": [abst["text"] + abst["section"]
                         for abst in record["body_text"]]}

In [24]:
d_paper_df = d_json_files.map(collect_text).to_dataframe()

In [25]:
d_paper_df.shape[0].compute()

33376

In [26]:
# collecting text from json papers
d_paper_df["text"] = d_paper_df.apply(lambda row: " ".join(row["abstract"]) + \
                                                           " ".join(row["body"]), meta=(None, 'object'), axis=1)

In [43]:
import numpy as np
import uuid

In [46]:
uuid.uuid4().__str__()

'67e82c72-6d7a-4618-b726-c7b9577e78af'

In [54]:
# creating a unique id
d_paper_df["id"] = d_paper_df.apply(lambda x: uuid.uuid4().__str__(), 
                                    meta=(None, 'object'), 
                                    axis=1)

In [56]:
# writing id and text to file
d_paper_df[["id", "text"]].to_bag()\
                                .map(lambda x: {"id": x[0], "text": x[1]})\
                                .map(json.dumps)\
                                .to_textfiles(
                '../data/covid19/pre-database-documents/processed/processed.*.json'
) 

['/Users/bajayi-obe/Documents/personal_git/cvd19-truth-finder/data/covid19/pre-database-documents/processed/processed.0.json',
 '/Users/bajayi-obe/Documents/personal_git/cvd19-truth-finder/data/covid19/pre-database-documents/processed/processed.1.json',
 '/Users/bajayi-obe/Documents/personal_git/cvd19-truth-finder/data/covid19/pre-database-documents/processed/processed.2.json',
 '/Users/bajayi-obe/Documents/personal_git/cvd19-truth-finder/data/covid19/pre-database-documents/processed/processed.3.json']

---

# Create an SQLitte Database from Pre-Processed Files

In [28]:
!python ../scripts/retriever/build_db.py \
"../data/covid19/pre-database-documents/processed" \
"../data/covid19/doc-db/full_doc.db" \
--num-workers=1

Traceback (most recent call last):
  File "../scripts/retriever/build_db.py", line 18, in <module>
    from drqa.retriever import utils
ModuleNotFoundError: No module named 'drqa'


# Create TF-IDF Model from Database

In [37]:
!python ../scripts/retriever/build_tfidf.py \
"../data/covid19/doc-db/full_doc.db" \
"../data/covid19/full_doc_ranker"

05/09/2020 02:16:47 PM: [ Counting words... ]
05/09/2020 02:16:47 PM: [ Mapping... ]
05/09/2020 02:16:47 PM: [ -------------------------Batch 1/4------------------------- ]
05/09/2020 02:16:47 PM: [ -------------------------Batch 2/4------------------------- ]
05/09/2020 02:16:47 PM: [ -------------------------Batch 3/4------------------------- ]
05/09/2020 02:16:47 PM: [ -------------------------Batch 4/4------------------------- ]
05/09/2020 02:16:47 PM: [ Creating sparse matrix... ]
05/09/2020 02:16:47 PM: [ Making tfidf vectors... ]
05/09/2020 02:16:49 PM: [ Getting word-doc frequencies... ]
05/09/2020 02:16:50 PM: [ Saving to /home/ubuntu/cvd19-truth-finder/data/covid19/doc_ranker/doc-tfidf-ngram=2-hash=16777216-tokenizer=simple.npz ]


---

# Reader

### Pre-Process SQUAD Dataset

#### Train set

In [55]:
!python ../scripts/reader/preprocess.py \
"/home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets" \
"/home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets" \
--split SQuAD-v1.1-train \
--tokenizer spacy

Loading dataset /home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets/SQuAD-v1.1-train.json
Will write to file /home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets/SQuAD-v1.1-train-processed-spacy.txt
Total time: 142.2876 (s)


#### dev set

In [57]:
!python ../scripts/reader/preprocess.py \
"/home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets" \
"/home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets" \
--split SQuAD-v1.1-dev \
--tokenizer spacy

Loading dataset /home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets/SQuAD-v1.1-dev.json
Will write to file /home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets/SQuAD-v1.1-dev-processed-spacy.txt
Total time: 21.2823 (s)


## Train Model

In [ ]:
!python ../scripts/reader/train.py \
--num-epochs=1 \
--model-dir="/home/ubuntu/cvd19-truth-finder/data/covid19/model" \
--data-dir="/home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets" \
--train-file="/home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets/SQuAD-v1.1-train-processed-spacy.txt" \
--dev-file="/home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets/SQuAD-v1.1-dev-processed-spacy.txt" \
--embed-dir="/home/ubuntu/cvd19-truth-finder/drQA-training-datasets/datasets/" \
--embedding-file="glove.6B.50d.txt"

# Running Full Pipeline

In [63]:
import torch
import argparse
import code
import prettytable

from termcolor import colored
from drqa import pipeline
from drqa.retriever import utils

if __name__ == '__main__':
    # Arguments
    args_reader_model="../data/covid19/model/20200509-c9d21e14.mdl"
    args_retriever_model="../data/covid19/full_doc_ranker/full_doc-tfidf-ngram=2-hash=16777216-tokenizer=simple.npz"
    args_doc_db="../data/covid19/doc-db/full_doc.db"
    args_tokenizer="spacy"
    args_candidate_file=None
    args_no_cuda=True
    args_gpu=-1

    args_cuda = not args_no_cuda and torch.cuda.is_available()
    if args_cuda:
        torch.cuda.set_device(args_gpu)

    if args_candidate_file:
        candidates = set()
        with open(args_candidate_file) as f:
            for line in f:
                line = utils.normalize(line.strip()).lower()
                candidates.add(line)
    else:
        candidates = None

    DrQA = pipeline.DrQA(
        cuda=args_cuda,
        fixed_candidates=candidates,
        reader_model=args_reader_model,
        ranker_config={'options': {'tfidf_path': args_retriever_model}},
        db_config={'options': {'db_path': args_doc_db}},
        tokenizer=args_tokenizer
    )

    # ------------------------------------------------------------------------------
    # Drop in to interactive mode
    # ------------------------------------------------------------------------------


    def process(question, candidates=None, top_n=5, n_docs=5):
        predictions = DrQA.process(
            question, candidates, top_n, n_docs, return_context=True
        )
        table = prettytable.PrettyTable(
            ['Rank', 'Answer', 'Doc', 'Answer Score', 'Doc Score']
        )
        for i, p in enumerate(predictions, 1):
            table.add_row([i, p['span'], p['doc_id'],
                           '%.5g' % p['span_score'],
                           '%.5g' % p['doc_score']])
        print('Top Predictions:')
        print(table)
        print('\nContexts:')
        for p in predictions:
            text = p['context']['text']
            start = p['context']['start']
            end = p['context']['end']
            output = (text[:start] +
                      colored(text[start: end], 'green', attrs=['bold']) +
                      text[end:])
            print('[ Doc = %s ]' % p['doc_id'])
            print(output + '\n')

    process("virus",n_docs=1000)

Top Predictions:
+------+--------+-----+--------------+-----------+
| Rank | Answer | Doc | Answer Score | Doc Score |
+------+--------+-----+--------------+-----------+
+------+--------+-----+--------------+-----------+

Contexts:
